In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('train_data.csv')

  id_colaborador;id_ultimo_jefe;seniority;modalidad_trabajo;distancia_oficina;dias_baja_salud;genero;canal_reclutamiento;permanencia_promedio;fecha_nacimiento;salario;performance_score;psi_score;fecha_incorporacion;estado_civil;abandono_6meses
0  100247;102074;1;Híbrida;1.76;1;Mujer;Ferias & ...                                                                                                                                                                                               
1  103355;102115;1;Híbrida;0.76;2;Hombre;Ferias &...                                                                                                                                                                                               
2  100669;102060;1;Híbrida;4.95;3;Mujer;Referidos...                                                                                                                                                                                               
3  103760;102062;1;Híbri